# Consolidate sea ice observations and calculate regional averages

### Author - Chris Wyburn-Powell, see the latest version on [github](https://github.com/chrisrwp/low-frequency-variability/blob/main/input_data/Regional_sea_ice_obs.ipynb)

**Input:**
- Raw observational datasets of sea ice concentration (HadISST1 and NSIDC CDR, BT, NT) and thickness (PIOMAS)

**Method:**
- Fill in any missing data or spurious values. 
- Create a mask for each observational dataset for the regions as per NSIDC MASIE
- Compute the regional average sea ice concentration/thickness, regional area/volume, and pan-Arctic sea ice area for each observational dataset
- Detrend and apply a 2 year lowpass Butterworth filter to the regional average SIC and SIT to use as inputs for training the ML model 

**Output:**
- Detrended and 2 year lowpass filtered regional average SIC and SIT as NetCDF files for each observational dataset

In [35]:
import xarray as xr
import numpy as np
import glob
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import struct
import scipy.signal as signal

In [3]:
#load dates in datetime64 for the mid-month date 1920-01 to 2014-12
time_seqential = xr.open_dataarray('/glade/work/cwpowell/'\
    +'low-frequency-variability/raw_data/datetime64_1850_2100_monthly.nc')

# NSIDC CDR
## Fill gaps to create homogenous NetCDF file 1979-2021
**Files downloaded from:**
`ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V3/north/monthly/seaice_conc_monthly_nh_n07_197901_v03r01.nc`
`ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V3/north/monthly/seaice_conc_monthly_nh_n07_198707_v03r01.nc`
`ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V3/north/monthly/seaice_conc_monthly_nh_f08_198708_v03r01.nc`
`ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V3/north/monthly/seaice_conc_monthly_nh_f08_199112_v03r01.nc`
`ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V3/north/monthly/seaice_conc_monthly_nh_f11_199201_v03r01.nc`
`ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V3/north/monthly/seaice_conc_monthly_nh_f11_199509_v03r01.nc`
`ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V3/north/monthly/seaice_conc_monthly_nh_f13_199510_v03r01.nc`
`ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V3/north/monthly/seaice_conc_monthly_nh_f13_200712_v03r01.nc`
`ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02202_V3/north/monthly/seaice_conc_monthly_nh_f17_200801_v03r01.nc`

In [3]:
#load all monthly files into a single xarray dataset and correct time dimension
#surpress warnings as the non-standard time produces an error, later overwritten

all_CDR_data = []

for file in glob.glob('/glade/work/cwpowell/low-frequency-variability/'\
                      +'raw_data/sea_ice_obs/NSIDC_CDR_monthly/'\
                      +'seaice_conc_monthly_nh*'):
    
    all_CDR_data.append(xr.open_dataset(file))
    
#concatenate all of the monthly files into a single xarray dataaray
all_CDR_xr = xr.concat((all_CDR_data), dim='tdim') 

#rename dimension so they match coordinates
all_CDR_xr = all_CDR_xr.rename({'tdim':'time', 'y':'ygrid', 'x':'xgrid'}) 

#sort by time dimension, files were loaded in a random order
all_CDR_xr = all_CDR_xr.sortby(all_CDR_xr['time']) 

#replace the time dimension with numpy.datetime64 objects for mid-month
all_CDR_xr['time'] = time_seqential.sel(time=slice('1979','2021'))

In [107]:
#set all non-sea ice to np.nan, exclude land/coastal grid points - values <2.5
CDR = all_CDR_xr['cdr_seaice_conc_monthly'].where(
    all_CDR_xr['cdr_seaice_conc_monthly']<1.1)

BT  = all_CDR_xr['nsidc_bt_seaice_conc_monthly'].where(
    all_CDR_xr['nsidc_bt_seaice_conc_monthly']<1.1) 

NT  = all_CDR_xr['nsidc_nt_seaice_conc_monthly'].where(
    all_CDR_xr['nsidc_nt_seaice_conc_monthly']<1.1) 


In [108]:
#for filling values of 1984-07, 1987-12 and 1988-01 the following years 
#(1985-07, 1988-12 and 1989-01) were found to be closest to the year with the
#missing values for other months of the year by looking at HadISST1 values
#and interpolaion is not desirable
filled = []

for data_var in [CDR, BT, NT]: #loop through all 3 datasets
    CDR_xr_1984_07 = data_var.sel(time='1985-07').copy()
    CDR_xr_1984_07['time'] = xr.DataArray(
        data = time_seqential.sel(time='1984-07').values, 
        coords={'time': time_seqential.sel(time='1984-07').values}, 
        dims=['time']
    )
    
    CDR_xr_1987_12 = data_var.sel(time='1988-12').copy()
    CDR_xr_1987_12['time'] = xr.DataArray(
        data = time_seqential.sel(time='1987-12').values, 
        coords={'time': time_seqential.sel(time='1987-12').values}, 
        dims=['time']
    )
    
    CDR_xr_1988_01 = data_var.sel(time='1989-01').copy()
    CDR_xr_1988_01['time'] = xr.DataArray(
        data = time_seqential.sel(time='1988-01').values, 
        coords={'time': time_seqential.sel(time='1988-01').values}, 
        dims=['time']
    )

    filled.append(xr.concat((data_var.sel(time=slice('1979-01','1984-06')), 
                             CDR_xr_1984_07, data_var.sel(
                                 time=slice('1984-08','1987-11')), 
                             CDR_xr_1987_12, CDR_xr_1988_01, data_var.sel(
                                 time=slice('1988-02','2020-12'))), 
                            dim='time'))

In [109]:
#save interpolated SIC to NetCDF
CDR_filled = xr.Dataset({'CDR':filled[0], 'BT':filled[1], 'NT':filled[2]})

CDR_filled.attrs = {
    'Description': 'Arctic sea ice concentration (SIC) from the Climate Data '\
        +'Record (CDR), NASA Team (NT) and NASA Boostrap (BT). All months '\
        +'1979-2020, missing data (1984-07, 1987-12, 1988-01) filled with '\
        +'data from the following years (1985-07, 1988-12, 1989-01) as the '\
        +'following year SIA is closer than the preceeding year SIA for the '\
        +'months with data adjacent to the missing months.', 
    'Units'      : 'fraction',
    'Timestamp'  : str(datetime.datetime.utcnow().strftime(
        "%H:%M UTC %a %Y-%m-%d")),
    'Data source': 'NOAA/NSIDC Climate Data Record of Passive Microwave Sea '\
        +'Ice Concentration, Version 4, doi:10.7265/efmz-2t65.',
    'Analysis'   : 'https://github.com/chrisrwp/low-frequency-variability/'\
        +'tree/main/input_data/Regional_sea_ice_obs.ipynb'
}

CDR_filled.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
                     +'raw_data/sea_ice_obs/SIC_CDR_BT_NT_79-21_filled.nc')


**Download the ancillary grid data** <br> 
`ftp://sidads.colorado.edu/DATASETS/NOAA/G02202_V4//ancillary/G02202-cdr-ancillary-nh.nc` <br><br>
**Regrid the ancillary grid data to the masie regions** <br>
`cdo remapnn,/glade/work/cwpowell/low_freq_sea_ice/Raw_data/G02202-cdr-ancillary-nh.nc /glade/work/cwpowell/low-frequency-variability/raw_data/masie_masks/masiemask_ims4km.nc /glade/work/cwpowell/low-frequency-variability/raw_data/masie_masks/masiemask_NSIDC_CDR.nc`

# HadISST1
## Reduce the dataset to 1979-2021 
**Also exclude to >30N and replace 2009-03 with 2007-03 and 2009-04 with 2008-04.** <br>
**N.B. March and April 2009 have a large drops in SIA and SIE (notably large negative SIC anomalies in Hudson Bay, Labrador Sea and sea of Okhotsk), this discontinutiy is not shown in other datasets.** <br>
**Data obtained from:** `https://www.metoffice.gov.uk/hadobs/hadisst/data/HadISST_ice.nc.gz`

In [3]:
HadISST1 = xr.open_dataset('/glade/work/cwpowell/low-frequency-variability/'\
                           +'raw_data/sea_ice_obs/HadISST_ice.nc')

#select 1979-2020 and the area above 30N
HadISST1_30N = HadISST1['sic'].sel(time=slice('1979','2021')).where(
    HadISST1['latitude']>30, drop=True) 

#adjust time to mid-month to exactly match other datasets
HadISST1_30N['time'] = time_seqential.sel(time=slice('1979','2021'))

HadISST1_30N.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
                        +'raw_data/sea_ice_obs/HadISST1_NH_79-21.nc')

In [4]:
#fill the spurious data with the most appropriate nearby year's data
HadISST1_2009_03 = HadISST1_30N.sel(time='2007-03').copy()
HadISST1_2009_03['time'] = xr.DataArray(
    data = time_seqential.sel(time='2009-03').values,                                
    coords={'time': time_seqential.sel(time='2009-03').values}, 
    dims=['time']
)

HadISST1_2009_04 = HadISST1_30N.sel(time='2008-04').copy()
HadISST1_2009_04['time'] = xr.DataArray(
    data = time_seqential.sel(time='2009-04').values,
    coords={'time': time_seqential.sel(time='2009-04').values}, 
    dims=['time']
)

HadISST1_30N_correct = xr.concat(
    (HadISST1_30N.sel(time=slice('1979','2009-02')), 
    HadISST1_2009_03, HadISST1_2009_04, 
    HadISST1_30N.sel(time=slice('2009-05','2020'))), dim='time'
)

HadISST1_30N_correct.attrs = {
    'Description': 'Arctic sea ice concentration (SIC) from the Hadley '\
        +'Centre Sea Ice and Sea Surface Temperature data set (HadISST1). All'\
        +'months 1979-2021, discontinuities for 2009-03 and 2009-04 are filled'\
        +'with 2007-03 and 2008-04 which are the years with the most similar'\
        +'data according to satellite records.', 
    'Units'      : 'fraction',
    'Timestamp'  : str(datetime.datetime.utcnow().strftime(
        "%H:%M UTC %a %Y-%m-%d")),
    'Data source': 'HadISST1, doi:10.1029/2002JD002670.',
    'Analysis'   : 'https://github.com/chrisrwp/low-frequency-variability/'\
        +'tree/main/input_data/Regional_sea_ice_obs.ipynb'
}

#save this corrected data to NetCDF
HadISST1_30N_correct.to_netcdf(
    '/glade/work/cwpowell/low-frequency-variability/raw_data/sea_ice_obs/'\
    +'HadISST1_NH_79-21_filled.nc')

**Create a regional mask from MASIE regions** <br>
`cdo remapnn,/glade/work/cwpowell/low-frequency-variability/raw_data/sea_ice_obs/HadISST1_NH_79-21_filled.nc /glade/work/cwpowell/low-frequency-variability/raw_data/masie_masks/masiemask_ims4km.nc /glade/work/cwpowell/low-frequency-variability/raw_data/masie_masks/masiemask_HadISST1.nc`

# Calculate Regional average SIC and SIA from SIC observational datasets

In [11]:
def regional_calc_obs(SIC_data, area_data, region_mask, 
                      lat_name, lon_name):
    '''
    Calculate regional SIA and SIC 1979-2021 observational data
    Regions based on NSIDC MASIE regions. 
    
    Parameters
    ----------
    SIC_data: xarray.Dataset
        3D matrix of all observational SIC data, with time, lat, lon dimensions
    area_data: xarray.DataArray
        2D martix of cell area by latitude and longitude
    region_mask: xarray.DataArray
        2D martix of cell area by latitude and longitude
    lat_name: string
        The name of the meridional dimension
    lon_name: string
        The name of the zonal dimension
    
    Returns
    ----------
        xarray.Dataset of regional SIA and SIC
    '''  
    
    #only do the calculation where there is sea ice
    SIC = SIC_data.where(SIC_data>0)
    #make sure only grid cells with ocean, not land are summed 
    area_data = area_data.where(SIC.isel(time=0))
    
    #calculate the pan-Arctic SIA
    pan_Arctic_SIA = (SIC * area_data).sum(lat_name).sum(lon_name)
    
    SIA_regions = []
    SIC_regions_av = []
    
    #calculate SIA and average SIC for each region
    for region_ in np.arange(1,17): 
        SIC_region = SIC.where((region_mask==region_))
        SIA_region = (SIC_region*area_data).sum(lat_name).sum(lon_name)

        area_region = area_data.where(region_mask==region_).sum()

        SIA_regions.append(SIA_region)
        SIC_regions_av.append(SIA_region / area_region)

    SIA_all_regions = xr.concat((SIA_regions),dim='region')
    SIC_all_regions = xr.concat((SIC_regions_av),dim='region')

    SIA_all_regions['region'] = np.arange(1,17)
    SIC_all_regions['region'] = np.arange(1,17)
    
    return(SIA_all_regions, SIC_all_regions, pan_Arctic_SIA)

In [30]:
#run the regional calculations for each observational dataset
SIC_data = xr.open_dataarray('/glade/work/cwpowell/low-frequency-variability/'\
                        +'raw_data/sea_ice_obs/HadISST1_NH_79-21_filled.nc')
                     # +'raw_data/sea_ice_obs/SIC_CDR_BT_NT_79-21_filled.nc')
                        

area_data = xr.open_dataset('/glade/work/cwpowell/low-frequency-variability/'\
                              +'raw_data/sea_ice_obs/HadISST1_gridarea.nc')
                              # +'raw_data/sea_ice_obs/NSIDC_CDR_gridarea.nc')
                             

#rename x and y if using NSIDC data
# area_data = area_data.rename({'x':'xgrid', 'y':'ygrid'})
    
region_mask = xr.open_dataset('/glade/work/cwpowell/low-frequency-variability/'\
                              +'raw_data/masie_masks/masiemask_HadISST1.nc')
                              # +'raw_data/masie_masks/masiemask_NSIDC_CDR.nc')

#rename x and y if using NSIDC data
# region_mask = region_mask.rename({'x':'xgrid', 'y':'ygrid'})
    
    
# obs_regional = regional_calc_obs(SIC_data['CDR'], area_data['cell_area'],
#                                  region_mask['regions'], 'ygrid', 'xgrid')

obs_regional = regional_calc_obs(SIC_data, area_data['cell_area'],
                                 region_mask['regions'], 
                                 'latitude', 'longitude')

In [31]:
#get SIA into million square km and change SIC from fraction to %
regional_ds = xr.Dataset({'SIA':obs_regional[0]/1e12, 
                          'SIC':obs_regional[1]*100, 
                          'pan-Arctic-SIA':obs_regional[2]/1e12
                         })

regional_ds.attrs = {
    'Description': 'Regional sea ice area (SIA), regional average sea ice '\
        +'concentration (SIC) and pan-Arctic sea ice area for the '\
        +'observational dataset the Hadley Centre Sea Ice and Sea Surface '\
        +'Temperature data set (HadISST1) for the period 1979-2021. Regions '\
        +'as defined for NSIDC MASIE-NH Version 1, doi:10.7265/N5GT5K3K.',
    'Data source': 'HadISST1, doi:10.1029/2002JD002670.',
     #    +'observational dataset National Oceanic and Atmospheric '\
     #    +'Administration (NOAA) and National Snow and Ice Data Center (NSIDC)'\
     #    +' Climate Data Record of Passive Microwave Sea Ice Concentration, '\
     #    +'Version 4 for the period 1979-2021 - Climate Data Record (CDR) '\
     #    +'algorithum. Regions as defined for NSIDC MASIE-NH Version 1,'\
     #    +'doi:10.7265/N5GT5K3K.',
     # 'Data source': 'NOAA/NSIDC Climate Data Record of Passive Microwave Sea '\
     #    +'Ice Concentration, Version 4, doi:10.7265/efmz-2t65.',
    
    'Timestamp'  : str(datetime.datetime.utcnow().strftime(
        "%H:%M UTC %a %Y-%m-%d")),
    'Analysis'   : 'https://github.com/chrisrwp/low-frequency-variability/'\
        +'tree/main/input_data/Regional_sea_ice_obs.ipynb'
}

regional_ds['SIA'].attrs = {'Description': 'Regional sea ice area in million '\
                           +'square km.'}
regional_ds['SIC'].attrs = {'Description': 'Regional average sea ice '\
                            +'concentration in %.'}
regional_ds['pan-Arctic-SIA'].attrs = {'Description': 'Pan-Arctic sea ice '\
                           +'area in million square km.'}

regional_ds.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
                      +'raw_data/sea_ice_obs/'\
                      +'HadISST1_Regional_average_SIC_1979_2021.nc')
                      # +'NSIDC_CDR_Regional_Average_SIC_1979_2021.nc')
                      


## Now detrend SIC data, make a 2 year lowpass filter, and standardize

In [33]:
def filt_lowpass(data_, sample_freq, cutoff, order, ax_n, detrend=False,
                 standard=False):
    '''
    Filter a time series using a lowpass Butterworth filter. 
    Uses scipy.signal.butter and scipy.signal.filtfilt
    
    Parameters
    ----------
    data_ : n dimensional xarray dataarray,
        For 1979-2020 this is an array of shape [42] 
    sample_freq: float,
        The sampling frequency of the input data, typically sample_freq=1 [year]
    cutoff: float,
        The fraction of the nyquist frequency (itself half the sampling 
        frequency). To filter with a 2-year lowpass filter with
        sample_freq=1 (year), cutoff=0.25
    order: int
        The order of the Butterworth filter, typically 4-6
    ax_n : int
        Which axis to do the filtering on (time)
    detrend: bool
        Whether to detrend the data with a linear trend
    standard: bool
        Whether to standardize the data after filtering

    Returns
    ----------
        numpy array of the same shape as the input data
    '''

    if detrend: #detrend the data first
        data_ = (data_ * 0) + signal.detrend(data=data_.fillna(0), axis=ax_n)

    b, a = signal.butter(order, cutoff, btype='lowpass') #low pass filter
    #apply the filter forward and backward along a given axis
    filtered = signal.filtfilt(b, a, data_, axis=ax_n) 

    filtered_xr = (data_ * 0) + filtered

    if standard: #standardize the data
        filtered_xr = (filtered_xr - filtered_xr.mean('time')) \
                      / filtered_xr.std('time')

    return(filtered_xr)

In [41]:
unfiltered = xr.open_dataset('/glade/work/cwpowell/low-frequency-variability/'\
    +'raw_data/sea_ice_obs/NSIDC_NT_Regional_Average_SIC_1979_2021.nc')
    # +'raw_data/sea_ice_obs/HadISST1_Regional_average_SIC_1979_2021.nc')

all_month_data = []

for month_ in np.arange(1,13):
    #select regional average SIC data for each month 
    unfiltered_month = unfiltered['SIC'].sel(
        time=unfiltered['time.month']==month_)

    #filter with a 2 year lowpass filter for each month
    all_month_data.append(filt_lowpass(unfiltered_month, 1, 0.25, 5, 1, 
                                       detrend=True, standard=False)
                         )

all_months_xr = xr.Dataset(
    {'SIC': xr.concat((all_month_data), dim='time').sortby('time')}
)

all_months_xr.attrs = {
    'Description': '2 year lowpass filter of linearly detrended '\
        +'regional average sea ice concentration (SIC) for the observational '\
        +'dataset National Oceanic and Atmospheric Administration (NOAA) and '\
        +'National Snow and Ice Data Center (NSIDC) Climate Data Record of '\
        +'Passive Microwave Sea Ice Concentration, Version 4 for the period '\
        +'1979-2021 - NASA Team (NT) algorithum. Regions as '\
        +'defined for NSIDC MASIE-NH Version 1,doi:10.7265/N5GT5K3K.',
        # +'regional average sea ice concentration (SIC) for the Hadley '\
        # +'Centre Sea Ice and Sea Surface Temperature data set (HadISST1) '\
        # +'for the period 1979-2021. Regions as defined for NSIDC MASIE-NH '\
        # +'Version 1, doi:10.7265/N5GT5K3K.',
    'Units'      : 'sea ice concentration, %',
    'Timestamp'  : str(datetime.datetime.utcnow().strftime(
        "%H:%M UTC %a %Y-%m-%d")),
    # 'Data source': 'HadISST1, doi:10.1029/2002JD002670.',
    'Data source': 'NOAA/NSIDC Climate Data Record of Passive Microwave Sea '\
        +'Ice Concentration, Version 4, doi:10.7265/efmz-2t65.',
    'Analysis'   : 'https://github.com/chrisrwp/low-fequency-variability/'\
        +'input_data/Regional_sea_ice_obs.ipynb'
}

all_months_xr.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
    +'input_data/Regional_SIC_detrended_lowpass_1979_2021_NSIDC_NT.nc')

# Sea ice thickness - PIOMAS

**Download the grid data** <br>
`https://raw.githubusercontent.com/robbiemallett/piomas_bin_reader/master/grids/latgrid.dat` <br>
`https://raw.githubusercontent.com/robbiemallett/piomas_bin_reader/master/grids/longrid.dat` <br><br>
**Download the yearly binary files** <br>
`https://pscfiles.apl.washington.edu/zhang/PIOMAS/data/v2.1/heff/heff.H1979.gz`

In [73]:
#adapted from 
#https://github.com/robbiemallett/piomas_bin_reader/blob/master/processor.py

grids = {}
dims = (120,360)

for i in ['lon','lat']:
    grid = np.array(pd.read_csv('/glade/work/cwpowell/'\
                    +'low-frequency-variability/raw_data/sea_ice_obs/'\
                    +f'PIOMAS_yearly_binary/{i}grid.dat', header=None, 
                        delim_whitespace=True)
                   )

    flat_grid = grid.ravel()
    shaped_grid = flat_grid.reshape(dims)
        
    grids[i] = shaped_grid

def process_piomas(year, end_month=False):
    
    binary_dir = '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
                 +f'sea_ice_obs/PIOMAS_yearly_binary/heff.H{year}'
    
    ############################################################
    # Read File
    with open(binary_dir, mode='rb') as file:
    
        fileContent = file.read()
        data = struct.unpack("f" * (len(fileContent)// 4), fileContent)
        
        ############################################################
        # Put it in a 3D array
        native_data = np.full((12,dims[0],dims[1]),np.nan)

        for month in range(1,13):
            if end_month==False or int(end_month) >= month:
                start = (month-1)*(dims[0]*dims[1])
                end = month*(dims[0]*dims[1])
                thickness_list = np.array(data[start:end])

                gridded = thickness_list.reshape(dims[0],dims[1])
                native_data[month-1,:,:] = gridded
            
    ############################################################
    #Output to xarray Dataset    
    ds = xr.Dataset(
        data_vars={'thickness':(['time','x','y'],native_data)},

        coords =  {'longitude':(['x','y'],grids['lon']),
                   'latitude':(['x','y'],grids['lat']),
                   'time':(['time'],time_seqential.sel(
                        time=slice(str(year)+'-01', str(year)+'-12')))
                  }
    )

    if end_month:
        ds = ds.sel(time=slice(str(year)+'-01', str(year)+'-'
                               +str(end_month).zfill(2)))
        
    return ds

In [74]:
#reoganize the binary data for each year into a single xarray.Dataset
#for saving to NetCDF

all_years_piomas = []
for year_ in np.arange(1979,2023):
    if year_ == 2022:
        end_month = '06' #set the ending month of the lastest year's data
    else:
        end_month = False
        
    all_years_piomas.append(process_piomas(year_, end_month))

piomas_xr = xr.concat((all_years_piomas),dim='time')

piomas_xr.attrs = {
    'Description': 'Monthly mean Pan-Arctic Ice-Ocean Modeling and '\
        +'Assimilation System (PIOMAS) sea ice thickness data on '\
        +'the native 360x120 curve-linear model grid for January 1979 to '\
        +'June 2022.',
    'Units'      : 'm',
    'Timestamp'  : str(datetime.datetime.utcnow().strftime(
        "%H:%M UTC %a %Y-%m-%d")),
    'Data source': 'University of Washington Polar Science Center - PIOMAS '\
        +'Arctic sea ice thickness, Version 2.1, '\
        +'doi:10.1175/1520-0493(2003)131<0845:MGSIWA>2.0.CO;2.',
    'Analysis'   : 'https://github.com/chrisrwp/low-frequency-variability/'\
        +'tree/main/input_data/Regional_sea_ice_obs.ipynb'
}

In [53]:
piomas_xr.to_netcdf('/glade/work/cwpowell/low-frequency-variability/raw_data/'\
                    +'sea_ice_obs/PIOMAS_SIT_1979_01_2022_06_native_grid.nc')

In [42]:
#try code from here for attempting to regrid
#https://raw.githubusercontent.com/andypbarrett/regrid_piomas/master/source/regrid.py